In [ ]:
!gdown --id 1GUmQpeqRBEZFsMZvPNCH9EFqRMVqURSX

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1GUmQpeqRBEZFsMZvPNCH9EFqRMVqURSX
To: /content/whisper.zip
100% 2.28M/2.28M [00:00<00:00, 220MB/s]


In [ ]:
!unzip /content/whisper.zip
!mv /content/whisper/* /content/

Archive:  /content/whisper.zip
  inflating: whisper/tokenizer.py    
  inflating: whisper/model.py        
  inflating: whisper/utils.py        
  inflating: whisper/assets/mel_filters.npz  
  inflating: whisper/timing.py       
  inflating: whisper/transcribe.py   
  inflating: whisper/decoding.py     
  inflating: whisper/harvard.wav     
  inflating: whisper/audio.py        
  inflating: whisper/assets/multilingual.tiktoken  
mv: cannot move '/content/whisper/assets' to '/content/assets': Directory not empty


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"halexhoang","key":"d06cacec067b3b68ac3a865b42e60cec"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download kynthesis/vivos-vietnamese-speech-corpus-for-asr

Dataset URL: https://www.kaggle.com/datasets/kynthesis/vivos-vietnamese-speech-corpus-for-asr
License(s): Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)
vivos-vietnamese-speech-corpus-for-asr.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/vivos-vietnamese-speech-corpus-for-asr.zip

Archive:  /content/vivos-vietnamese-speech-corpus-for-asr.zip
replace vivos/COPYING? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: vivos/COPYING           
replace vivos/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace vivos/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import urllib.request
from pathlib import Path
# https://github.com/openai/whisper/blob/main/whisper/__init__.py
url ="https://openaipublic.azureedge.net/main/whisper/models/e5b1a55b89c1367dacf97e3e19bfd829a01529dbfdeefa8caeb59b3f1b81dadb/large-v3.pt"
model_file = Path("tiny.pt")
urllib.request.urlretrieve(url, model_file)

(PosixPath('tiny.pt'), <http.client.HTTPMessage at 0x7a93a7437530>)

In [ ]:
import torch
from pathlib import Path
from model import Whisper, ModelDimensions
import audio

In [ ]:
model_file = Path("tiny.pt")
checkpoint = torch.load(model_file, map_location="cpu")
dims = ModelDimensions(**checkpoint["dims"])

model = Whisper(dims)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

def print_shape_hook(name):
    def hook(module, input, output):
        if isinstance(output, tuple):
            output = output[0]
        print(f"{name}: {output.shape}")
    return hook

# encoder layers
for i, block in enumerate(model.encoder.blocks):
    block.register_forward_hook(print_shape_hook(f"Encoder block {i}"))

# decoder layers
for i, block in enumerate(model.decoder.blocks):
    block.register_forward_hook(print_shape_hook(f"Decoder block {i}"))
audio_path = "/content/vivos/test/waves/VIVOSDEV01/VIVOSDEV01_R002.wav"

In [ ]:
audio_array = audio.load_audio("harvard.wav")
print(audio_array.shape)
# Đây là vector 1D chứa toàn bộ các mẫu sóng (waveform) của file harvard.wav.
# 293699 = số mẫu (samples).
# Vì Whisper đã resample về 16kHz, tức là mỗi giây có 16,000 mẫu.
# → file của bạn dài khoảng 18.3 giây.
# Ý nghĩa: mỗi phần tử trong vector là cường độ âm thanh tại một thời điểm (float32, thường trong [-1, 1]).
# hình ảnh rời rạc của sóng âm liên tục trong ~18.3 giây.

(293699,)


In [ ]:
mel = audio.log_mel_spectrogram(audio_array)
print(mel.shape)
# Đây là kết quả sau khi biến waveform thành ảnh phổ tần số.

# Mỗi phần tử trong ma trận là một giá trị năng lượng (energy) sau khi log-transform.
# Chỉ số hàng (0–79) = filter Mel (dải tần số).
# Chỉ số cột (0–1834) = frame thời gian (mỗi frame ≈ 10ms).
# Giá trị thường là số âm (vì đã log) → càng lớn (ít âm) thì năng lượng tại tần số đó càng mạnh.

# Bạn có thể coi spectrogram như một bức ảnh:
# Trục X: thời gian (1835 cột, ~18.3 giây).
# Trục Y: tần số (80 bands).
# Pixel: độ sáng = năng lượng ở tần số đó, tại thời điểm đó.

torch.Size([80, 1835])


In [ ]:
result = model.transcribe(audio_path)
# Audio waveform → Spectrogram → Encoder [1, 1500, 384]
# → Cross Attention với Decoder [1, T, 384]
# → Linear Projection [1, T, 51865]
# → Softmax → chọn token
# → Ghép lại thành text

Encoder block 0: torch.Size([1, 1500, 1280])
Encoder block 1: torch.Size([1, 1500, 1280])
Encoder block 2: torch.Size([1, 1500, 1280])
Encoder block 3: torch.Size([1, 1500, 1280])
Encoder block 4: torch.Size([1, 1500, 1280])
Encoder block 5: torch.Size([1, 1500, 1280])
Encoder block 6: torch.Size([1, 1500, 1280])
Encoder block 7: torch.Size([1, 1500, 1280])
Encoder block 8: torch.Size([1, 1500, 1280])
Encoder block 9: torch.Size([1, 1500, 1280])
Encoder block 10: torch.Size([1, 1500, 1280])
Encoder block 11: torch.Size([1, 1500, 1280])
Encoder block 12: torch.Size([1, 1500, 1280])
Encoder block 13: torch.Size([1, 1500, 1280])
Encoder block 14: torch.Size([1, 1500, 1280])
Encoder block 15: torch.Size([1, 1500, 1280])
Encoder block 16: torch.Size([1, 1500, 1280])
Encoder block 17: torch.Size([1, 1500, 1280])
Encoder block 18: torch.Size([1, 1500, 1280])
Encoder block 19: torch.Size([1, 1500, 1280])
Encoder block 20: torch.Size([1, 1500, 1280])
Encoder block 21: torch.Size([1, 1500, 1280]

In [ ]:
print("Transcript:", result["text"])

Transcript:  Tiến cọc cạch khẩn lại của những góp sắt


In [ ]:
!pip install jiwer

In [ ]:
import os
from tqdm import tqdm
from jiwer import wer

VIVOS_TEST_PATH = "/content/vivos/test"
PROMPTS_PATH = os.path.join(VIVOS_TEST_PATH, "prompts.txt")
WAVES_PATH = os.path.join(VIVOS_TEST_PATH, "waves")

model_file = Path("tiny.pt")
checkpoint = torch.load(model_file, map_location="cpu")
dims = ModelDimensions(**checkpoint["dims"])

model = Whisper(dims)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

#prompts.txt (ID -> text)
id2text = {}
with open(PROMPTS_PATH, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split(" ", 1)
        if len(parts) == 2:
            file_id, text = parts
            id2text[file_id] = text.lower()

references, predictions = [], []

for spk_folder in os.listdir(WAVES_PATH):
    spk_path = os.path.join(WAVES_PATH, spk_folder)
    if not os.path.isdir(spk_path):
        continue

    for wav_file in tqdm(os.listdir(spk_path)):
        if not wav_file.endswith(".wav"):
            continue

        file_id = wav_file.replace(".wav", "")
        wav_path = os.path.join(spk_path, wav_file)


        result = model.transcribe(wav_path, language="vi", fp16=False)
        pred_text = result["text"].lower().strip()

        if file_id in id2text:
            references.append(id2text[file_id])
            predictions.append(pred_text)

error = wer(references, predictions)
accuracy = 1 - error

print(f"\nWord Error Rate (WER): {error:.3f}")
print(f"Accuracy (1 - WER): {accuracy:.3f}")


100%|██████████| 40/40 [00:53<00:00,  1.33s/it]


Word Error Rate (WER): 0.158
Accuracy (1 - WER): 0.842
